<a href="https://colab.research.google.com/github/KennyThinh/pytorch/blob/main/08_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
from torch import optim
from torchvision import transforms, datasets
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(in_features=kwargs["input_shape"], out_features=128)
        self.encoder_output_layer = nn.Linear(in_features=128, out_features=128)
        self.decoder_hidden_layer = nn.Linear(in_features=128, out_features=128)
        self.decoder_output_layer = nn.Linear(in_features=128, out_features=kwargs["input_shape"])

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

In [9]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=784).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [13]:

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root="~/torch_datasets", train=True, transform=transform, download=True)

test_dataset = datasets.MNIST(root="~/torch_datasets", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /root/torch_datasets/MNIST/raw/train-images-idx3-ubyte.gz to /root/torch_datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /root/torch_datasets/MNIST/raw/train-labels-idx1-ubyte.gz to /root/torch_datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /root/torch_datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/torch_datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /root/torch_datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/torch_datasets/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid poten

In [16]:
epochs=30
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch : 1/30, loss = 0.016550
epoch : 2/30, loss = 0.014567
epoch : 3/30, loss = 0.013610
epoch : 4/30, loss = 0.013025
epoch : 5/30, loss = 0.012602
epoch : 6/30, loss = 0.012229
epoch : 7/30, loss = 0.011602
epoch : 8/30, loss = 0.011338
epoch : 9/30, loss = 0.011140
epoch : 10/30, loss = 0.010981
epoch : 11/30, loss = 0.010857
epoch : 12/30, loss = 0.010744
epoch : 13/30, loss = 0.010649
epoch : 14/30, loss = 0.010569
epoch : 15/30, loss = 0.010486
epoch : 16/30, loss = 0.010422
epoch : 17/30, loss = 0.010352
epoch : 18/30, loss = 0.010295
epoch : 19/30, loss = 0.010225
epoch : 20/30, loss = 0.010165
epoch : 21/30, loss = 0.010112
epoch : 22/30, loss = 0.010059
epoch : 23/30, loss = 0.010022
epoch : 24/30, loss = 0.009981
epoch : 25/30, loss = 0.009950
epoch : 26/30, loss = 0.009926
epoch : 27/30, loss = 0.009893
epoch : 28/30, loss = 0.009855
epoch : 29/30, loss = 0.009844
epoch : 30/30, loss = 0.009810


**AutoEncoder for 2D**

In [ ]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        #Encoder Layers
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 4, kernel_size = 3, padding = 1)
        #Decoder Layers
        self.t_conv1 = nn.ConvTranspose2d(in_channels = 4, out_channels = 16, kernel_size = 2, stride = 2)
        self.t_conv2 = nn.ConvTranspose2d(in_channels = 16, out_channels = 1, kernel_size = 2, stride = 2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #コメントに28×28のモノクロ画像をi枚を入力した時の次元を示す
        #encode#                          #in  [i, 1, 28, 28] 
        x = self.relu(self.conv1(x))      #out [i, 16, 28, 28]  
        x = self.pool(x)                  #out [i, 16, 14, 14]
        x = self.relu(self.conv2(x))      #out [i, 4, 14, 14]
        x = self.pool(x)                  #out [i ,4, 7, 7]
        #decode#
        x = self.relu(self.t_conv1(x))    #out [i, 16, 14, 14]
        x = self.sigmoid(self.t_conv2(x)) #out [i, 1, 28, 28]
        return x

In [ ]:
epochs=30
model = ConvAutoencoder()
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))